# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Zadanie domowe 2

In [1]:
import pandas as pd
import numpy as np 

## Zadanie 1 - eksploracja przestrzeni zagnieżdżeń
Wczytajmy do przestrzeni plik zagnieżdżeń, który należy pobrać ze strony:
https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pl.vec Są to zagnieżdzenia dla języka polskiego uzyskane systemem fastText.

Do przestrzeni wczytujemy tylko 100 tys. najczęstrzych słów, tak aby operacje przebiegały szybciej.

In [34]:
import io
def load_vectors(fname, limit = 100000):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    n = min(n,limit)
    embeddings = np.empty((n,d), dtype = np.float)
    words_idx = []
    for i, line in enumerate(fin):
        if i >= limit:
            break
        tokens = line.rstrip().split(' ')
        words_idx.append(tokens[0])
        embeddings[i] =  np.array(tokens[1:]).astype(np.float)
    return words_idx, embeddings
words_idx, embeddings = load_vectors('wiki.pl.vec')

Poniższe zadania mają na celu poekserymentowanie z przestrzenią zagnieżdżeń, ale też zrozumienie stojącymi za nich operacji. Dozwolone jest korzystanie tylko z podstawowych operatorów Python i numpy (w szczególności zakaz dotyczy: sklearn, gensim, fasttext itd.)

Jeśli potrzebujesz do dalszego przetwarzania przeprowadzenia jakichś normalizacji macierzy -- możesz wstępnie przetworzyć macierz zagnieżdzeń poniżej. Pamiętaj, że sprawdzarka będzie używała wywołań do `embeddings` (i `words_idx`) -- musisz nadpisać macierz zagnieżdzeń. To pole jest pomocnicze i nie podlega ocenie.

In [35]:
wrd_embedding = {w: embeddings[i] for i,w in enumerate(words_idx)}

Zaimplementuj funkcję, która obliczy podobieństwo kosinusowe pomiędzy dwoma wyrazami.

In [4]:
def calc_sim(word_a, word_b, words_idx, embeddings):
    emb_a, emb_b = wrd_embedding[word_a], wrd_embedding[word_b]
    return np.dot(emb_a, emb_b)/((np.sum(emb_a**2)**.5)*(np.sum(emb_b**2)**.5))

def calc_sim_embds(emb_a, emb_b, words_idx, embeddings):
    return np.dot(emb_a, emb_b)/((np.sum(emb_a**2)**.5)*(np.sum(emb_b**2)**.5))

In [5]:
from nose.tools import assert_almost_equal
assert_almost_equal(calc_sim("bieber", "rihanna", words_idx, embeddings), calc_sim("rihanna", "bieber", words_idx, embeddings))

Policz podobieństwo pomiędzy wyrazem `bieber` a wyrazami:
    - `rihanna`
    - `piłsudski`
    - `kanada`
    - `polska`
    - `piosenka`

In [6]:
calc_sim("bieber", "rihanna", words_idx, embeddings)

0.524583248263655

In [7]:
for wrd in ['rihanna', 'piłsudski','kanada','polska','piosenka']:
    print("{} - {}: {}".format('bieber',wrd, calc_sim("bieber", wrd, words_idx, embeddings)))

bieber - rihanna: 0.524583248263655
bieber - piłsudski: 0.19303958051463557
bieber - kanada: 0.20042742126487928
bieber - polska: 0.12505934735679378
bieber - piosenka: 0.28748713688583316


Zaimplementuj funkcję, która zwróci najbardziej podobne słowa (miara kosinusowa) do danego słowa `word`. W wyniku wypisz tylko `top` słów z najbliższymi zagnieżdżeniami, pomijając słowo `word`.

In [8]:
def find_similar(word, words_idx, embedding_matrix, top=10):
    similar_words = {}
    for i, other_word in enumerate(words_idx):
        if other_word!=word:
            similar_words[other_word] = calc_sim(word, other_word, words_idx, embedding_matrix)
    return sorted(similar_words.keys(), key=lambda x: similar_words[x], reverse=True)[:top]

In [9]:
assert len(find_similar("radość", words_idx, embeddings)) == 10

Znajdź najbardziej podobne słowa do kobieta, politechnika, mateusz, szczecin, niemcy, piłsudski

In [10]:
find_similar("kobieta", words_idx, embeddings)

['kobietą',
 'dziewczyna',
 'mężczyzna',
 'kobietę',
 'dziewczynka',
 'mężczyznę',
 'staruszka',
 'mężczyzną',
 'kobiecie',
 'mężczyzny']

In [37]:
for w in ['kobieta', 'politechnika', 'mateusz', 'szczecin', 'niemcy', 'piłsudski']:
    print(w +' :')
    print(', '.join(find_similar(w, words_idx, embeddings)), end='\n\n')

kobieta :
kobietą, dziewczyna, mężczyzna, kobietę, dziewczynka, mężczyznę, staruszka, mężczyzną, kobiecie, mężczyzny

politechnika :
politechniki, politechniką, politechnikę, politechniczny, politechnice, politechnicznej, politechnicznego, politechnicznym, inżynierska, elektrotechnika

mateusz :
łukasz, bartłomiej, bartosz, kacper, marcin, mateusza, tomasz, patryk, rafał, mateuszem

szczecin :
szczecinek, szczeciński, szczecinem, gryfino, szczecinie, stargard, szczecina, koszalin, szczecińska, świnoujście

niemcy :
niemieccy, naziści, alianci, okupanci, polacy, hitlerowcy, niemieckie, rosjanie, niemców, niemcom

piłsudski :
piłsudskim, piłsudskiego, piłsudskiemu, sosnkowski, mościcki, śmigły, józef, żeligowski, piłsudczyków, sosnkowskiego



Krótko skomentuj wyniki dla słowa `niemcy`. Które z powstałych analogii biorą się z semantycznego powiązania a które z semantycznego podobieństwa?

Słowo `niemcy` otrzymały zarówno słowa związane z narodowością, jak i również związane z znaczeniem historycznym. Najprawdododobniej korpus języka, użyty do wytrenowania zagnieżdżeń, zawierał również zbiory traktujące o historii polski, w której to WWII pełni bardzo istotną rolę.

Semantycznie słowo `niemcy` powiązane jest z takimi słowami jak 'naziści', 'okupanci', 'hitlerowcy', z którymi najprawdopodobniej w korpusie używane były zamiennie. Semantycznie podobne jest do słów 'alianci', 'polacy', 'rosjanie'.

Zaimplementuj funkcje szukającą brakującego elementu relacji ,,`word_a` jest do `word_a2` jak `word_b` jest do...''. Funkcja powinna zwrócić 10 najbardziej pasujących słow z pominięciem słów będących jej argumentami.

In [12]:
def find_similar_pair(word_a, word_a2, word_b,  words_idx, matrix, top=10):
    emb_a = wrd_embedding[word_a]
    emb_a2 = wrd_embedding[word_a2]
    emb_b = wrd_embedding[word_b]
    nw_emb = emb_a2 - emb_a + emb_b
    
    similar_words = {}
    for other_word in words_idx:
        if other_word!= word_a and other_word!= word_a2 and other_word!= word_b:
            similar_words[other_word] = calc_sim_embds(nw_emb, wrd_embedding[other_word], words_idx, matrix)
    return sorted(similar_words.keys(), key=lambda x: similar_words[x], reverse=True)[:top]

In [13]:
assert find_similar_pair( "mężczyzna", "król", "kobieta", words_idx, embeddings)[0] == "królowa"

Pieniądze są do profesora jak wiedza do...

In [14]:
find_similar_pair( "pieniądze", "profesor", "wiedza", words_idx, embeddings)

['habilitowany',
 'docent',
 'wykładowca',
 'profesorem',
 'habilitacja',
 'adiunkt',
 'rektor',
 'profesora',
 'humanistycznych',
 'naukowiec']

Mateusza jest do mateusz jak łukasza do ...

In [15]:
find_similar_pair( "mateusza", "mateusz", "łukasza", words_idx, embeddings)

['łukasz',
 'bartłomiej',
 'bartosz',
 'maciej',
 'tomasz',
 'rafał',
 'patryk',
 'marcin',
 'michał',
 'przemysław']

Warszawa jest do "polska" jak "berlin" do ...

In [16]:
find_similar_pair( "warszawa", "polska", "berlin", words_idx, embeddings)

['niemiecka',
 'berliner',
 'wschodnioniemiecka',
 'berlińska',
 'deutschland',
 'deutsche',
 'brandenburg',
 'berlinem',
 'germany',
 'niemcy']

Zurich jest do ETH jak Poznań do ...

In [17]:
find_similar_pair( "zurich", "eth", "poznań", words_idx, embeddings)

['„poznań',
 'wrocław',
 'poznania',
 'poznańskie',
 'uam',
 'poznaniu',
 'kraków',
 'gniezno',
 'poznańską',
 'wlkp']

Niemcy są do Merkel jak Polska do ...

In [18]:
find_similar_pair( "niemcy", "merkel", "polska", words_idx, embeddings)

['kaczyńska',
 'lewandowska',
 'kwaśniewska',
 'ekonomistka',
 'lekarka',
 'parlamentarzystka',
 'marcinkiewicz',
 'olszewska',
 'bezpartyjna',
 'dziennikarka']

Na wektorach możemy wykonywać standardowe operacje algebry liniowej takie jak np. projekcja czyli rzutowanie danych na jakichś zbiór osi (więcej: notatki z algebry liniowej np. https://ocw.mit.edu/courses/mathematics/18-06sc-linear-algebra-fall-2011/least-squares-determinants-and-eigenvalues/projections-onto-subspaces/). W szczególności może to się przydać do zrzutowania słowa na przestrzeń w której pewny wybrany kierunek (wskazywany przez wektor) jest eliminowany.

Do czego może to się przydać? Jeśli uruchomisz funkcję `find_similar` dla słowa ,,mateusza'' znajdziesz m.in. ,,łukasza'' ale także ,,ewangelia'', ,,ewangelisty'' i ,,apostoła''. Chcąc pominąc kontekst religijny tego słowa możesz zrzutować jego reprezentacje na przestrzeń bez wektora ,,ewangelia'' i poszukać jego najbliższych sąsiadów (którymi będą teraz po prostu imiona męskie). Zaimplementuj taką funkcję.


In [19]:
def projection(vec_a, vec_b):
    return vec_a*(vec_a.T.dot(vec_b))/vec_a.T.dot(vec_a)

def find_similar_with_rejection(word, remove, words_idx, matrix, top=10):
    """
    Działanie analogiczne do find_similar z dodatkowym parametrem remove, 
    który jest *listą* słów, które należy wyrzucić poprzez projekcję.
    Dla remove=[] powinno się zwracać dokładnie to samo co find_similar
    """
    emb = wrd_embedding[word].copy()
    for r in remove:
        emb = emb - projection(wrd_embedding[r], emb)
        
    similar_words = {}
    for other_word in words_idx:
        if other_word!= word:
            othr_emb = wrd_embedding[other_word].copy()
            similar_words[other_word] = calc_sim_embds(emb, othr_emb , words_idx, matrix)
    return sorted(similar_words.keys(), key=lambda x: similar_words[x], reverse=True)[:top]
    
print ("Standardowe poszukiwanie:", find_similar_with_rejection("mateusza",[] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings))

Standardowe poszukiwanie: ['łukasza', 'ewangelii', 'ewangelisty', 'ewangelia', 'bartłomieja', 'ewangeliach', 'apostoła', 'mateusz', 'tymoteusza', 'jakuba']
Poszukiwanie po projekcji: ['macieja', 'bartłomieja', 'marcina', 'łukasza', 'andrzeja', 'piotra', 'jakuba', 'antoniego', 'michała', 'tomasza']


In [20]:
assert "ewangelii" in find_similar_with_rejection("mateusza",[] , words_idx, embeddings)
assert "ewangelii" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)
assert "ewangelisty" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)


Analogicznie słowo ,,java'' jest nie tylko nazwą języka programownia (https://pl.wikipedia.org/wiki/Java_(ujednoznacznienie)) -- jest np. nazwą geograficzną (indonezyjska wyspa koło Sumatry). Sprawdź jakie wyrazy są podobne do "java" oraz po odrzuceniu kierunku "javascript" (tj. kierunku związanego z językami programowania).

In [21]:
print("Dla słowa java najbardziej podobne słowa to:\n", 
      ', '.join(find_similar_with_rejection("java",[] , words_idx, embeddings)), end='\n\n')
print("Po dokonaniu odrzucenia kierunku do słowa javascript:\n",
      ', '.join(find_similar_with_rejection("java",['javascript'] , words_idx, embeddings)), end='\n\n')

Dla słowa java najbardziej podobne słowa to:
 javascript, javy, c#, c++, programowania, implementacja, lisp, framework, api, programistyczne

Po dokonaniu odrzucenia kierunku do słowa javascript:
 sumatra, tromp, krążowniki, indonezja, niszczycielami, penang, niszczyciele, krążowników, hmas, jawa



Spróbuj poekseprymentować samemu!

In [22]:
print("Dla słowa java najbardziej podobne słowa to:\n", 
      ', '.join(find_similar_with_rejection("nauka",[] , words_idx, embeddings)), end='\n\n')
print("Po dokonaniu odrzucenia kierunku do słowa javascript:\n",
      ', '.join(find_similar_with_rejection("nauka",['nauka'] , words_idx, embeddings)), end='\n\n')
print("Interesujące")

Dla słowa java najbardziej podobne słowa to:
 „nauka, nauką, filozofia, etyka, teologia, nauki, naukach, edukacja, pedagogika, astrologia

Po dokonaniu odrzucenia kierunku do słowa javascript:
 kappa, budokan, ninja, dolnym, wieloetniczną, aquarium, ino, prefekturze, aso, pomocniczego

Interesujące


Wykonanie projekcji w przestrzeni zagnieżdżeń może być jedną z prostych technik zwalczenia tzw. gender bias (http://wordbias.umiacs.umd.edu/) w reprezentacji słów. Okazuje się, że wykonanie projekcji macierzy zagnieżdżeń na przestrzeń w której ,,brakuje kierunku he-she'' może być bardzo prostą techniką zredukowania tego typu obciążenia.

## Zadanie 2 - zagnieżdżenia dokumentów
W tym ćwiczeniu powócimy do zbioru tweetów, który analizowaliśmy w poprzednim dokumencie.

In [23]:
from helpers import DataSet
training_set = DataSet(['tweets.txt'])

Reading data set ['tweets.txt']


In [ ]:
for i in training_set.tweets:
    print(i.text)
    print(i.tokens)
    print(i.clazz)
    break

Tym razem do zbudowania reprezentacji będziemy używać narzędzie Universal Sentence Encoder stworzone przez Googla na bazie głębokiej sieci uśredniającej (i architektur rekurencyjnych). Poniższy kod pokazuje sposób użycia tego narzędzia. 
Kod spokojnie można wywoływać na CPU -- choć ściąganie modelu trochę może potrwać.

In [28]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])
print (embeddings)

tf.Tensor(
[[-0.03133016 -0.06338634 -0.01607501 ... -0.0324278  -0.04575741
   0.05370457]
 [ 0.05080863 -0.0165243   0.01573782 ...  0.00976661  0.03170121
   0.01788118]], shape=(2, 512), dtype=float32)


Wykorzystując reprezetnację USE wytrenuj wybrany klasyfikator z pakietu `sklearn` i zweryfikuj jego jakość działania.

In [29]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split

target_val = {'negative': -1, 'neutral': 0, 'positive':1}

def embdd_cls(training_set):
    X_train, X_test, y_train, y_test = train_test_split(
            [sample.text for sample in training_set.tweets],
            [target_val[sample.clazz] for sample in training_set.tweets],
            test_size=.2, random_state=1337)

    X_train_embdd = embed(X_train)
    X_test_embdd = embed(X_test)

    clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter_no_change=30)
    clf.fit(X_train_embdd, y_train)

    pred = clf.predict(X_test_embdd)

    precision, recall, fscore, support = precision_recall_fscore_support(y_test, pred, average='macro')
    acc = accuracy_score(y_test, pred)
    
    return precision, recall, fscore, acc


data = [embdd_cls(training_set) for n_features in range(10)]
precision = [e[0] for e in data]
recall = [e[1] for e in data]
fscore = [e[2] for e in data]
acc = [e[3] for e in data]
print("""Expected:
Precision: {}
Recall: {}
F-score: {}
Accuracy: {}""".format(np.mean(precision), np.mean(recall), np.mean(fscore), np.mean(acc)))

Expected:
Precision: 0.6326400755343269
Recall: 0.498728654476767
F-score: 0.5057007110095565
Accuracy: 0.6328754578754578


Skomentuj wyniki i odnieś je do wyników z poprzedniego zadania domowego. Na ile użycie reprezentacji rozproszonych pozwoliło na poprawę wyników?

Podejście USE pozwoliło uzyskać lepszy `accuracy` i `precision` niż dwie metody z poprzedniego zadania. Porównanie wyników odbyło się dla takiego samego parametru `alpha`, ale eksperymenty pokazały, że dostrojenie tego parametru pozwala na osiągnięcie jeszcze lepszych wyników. Mimo tego, że pojedyncza próbka reprezentowana jest przez zaledwie 512 cech, forma ta dużą siłę wyrazu, jako że powstała na drodze modelowania języka metodami automatycznymi, potrafiącymi z danych wyciągnąć wiedzę o tym, jak silnie poszczególne koncepty  są w języku i używa dostępnej, niskowymiarowej, reprezentacji w sposób bardziej optymalny niż pozostałe metody.

# Zadanie 3 - konstruowanie zagnieżdżeń
W tym ćwiczeniu kontynuujemy pracę z tweetami, ale pomijamy całkowicie ich klasy. Zbiór tweetów potraktujemy jako korpus do nauczenia zagnieżdżeń słów przy pomocy macierzy PMI.
- Wypełnij macierz kontekst - dokument przy użyciu symetrycznego okna o promieniu 4 (po 4 słowa w każdą stronę)
- Możesz ograniczyć słownictwo do 10K słów
- Przekształć macierz w macierz PPMI
- Stwórz zagnieżdżenia wykorzystując dekompozycję SVD do wybranej wymiarowości $d$ (ze względu na koszt obliczeniowy może to być mała wymiarowość np. $d=10$)

In [30]:
from sklearn.decomposition import TruncatedSVD
wrd_ctr = {}
unique_words = []
corpus = []
ctxt_ctr = 0
ctr = 0
for i in training_set.tweets:
    corpus.append(np.array(i.tokens))
    for t in i.tokens:
        ctr += 1
        if wrd_ctr.get(t) is not None:
            wrd_ctr[t] += 1
        else:
            wrd_ctr[t] = 1
            unique_words.append(t)
            
wrd_idx = dict(zip(unique_words, range(len(unique_words))))
            
matrix = np.zeros([len(unique_words)]*2)

def crawling_window(tokens, radius=4):
    ctxt_len = 0
    for i in range(len(tokens)):
        ctxt = tokens[max(0,i-radius):i] + tokens[(i+1):min(i+1,len(tokens))]
        ctxt_len+=len(ctxt)
        for c in ctxt:
            matrix[wrd_idx[tokens[i]]][wrd_idx[c]]+=1
    return ctxt_len
    
for i in training_set.tweets:
    ctxt_ctr+=crawling_window(i.tokens)
    
matrix = matrix/ctxt_ctr

freq_vec = np.array([list([wrd_ctr[wrd] for wrd, _ in wrd_idx.items()])])/ctr
matrix = matrix/(freq_vec*freq_vec.T)
matrix = np.log2(matrix)
matrix[matrix<0]=0

svd = TruncatedSVD(n_components=10, n_iter=17, random_state=47)
embedds = svd.fit_transform(matrix)
embedds.shape

/home/imegirin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log2


(16229, 10)

Przetestuj działanie Twoich zagnieżdżeń wykorzystując funkcję `find_similar` na wybranych słowach.

In [31]:
wrd_embedding= {wrd: embedds[i] for wrd, i in wrd_idx.items()}

def calc_sim(word_a, word_b, words_idx, embeddings):
    emb_a, emb_b = wrd_embedding[word_a], wrd_embedding[word_b]
    return np.dot(emb_a, emb_b)/((np.sum(emb_a**2)**.5)*(np.sum(emb_b**2)**.5))

def find_similar(word, words_idx, embedding_matrix, top=10):
    similar_words = {}
    for other_word in wrd_embedding.keys():
        if other_word!=word:
            similar_words[other_word] = calc_sim(word, other_word, words_idx, embedding_matrix)
    return sorted(similar_words.keys(), key=lambda x: similar_words[x], reverse=True)[:top]

print("Linux:")
print(find_similar("linux", words_idx, embedds), end="\n\n")
print("Market:")
print(find_similar("market", words_idx, embedds), end="\n\n")

Linux:
['universal', 'attempting', 'reinstall', 'lasting', 'capri', '^kw', 'worried', 'operating', 'playlist', '#windows10']

Market:


/home/imegirin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


['stadium', 'main', 'adrian', '#mfclive', 'also', 'meetup', 'drive', 'news', ':p', 'era']

